# Rappels jour 2 

* gotchas sur les fonctions 
    * pas d'arguments mutables en entré
    * accès aux varaible : L(ocal) E(nglobing) G(lobal) B(uiltin)
    * les variables englobing sont passées par closure
    * en python les closures : 
        * sont late binding 
        * sont en lecture seule
* curryfication =>  passer d'une fonction à N arguments à N fonctions imbriquées à 1 argument chacune 
* décorateurs
    * on peut le comprendre comme une composée de fonction `toto = deco(toto)` 
    * on peut chainer les décorateurs, l'ordre a une importance 
    * la fonction a l'intérieur s'appelle `wrapper` par convention 
    * dans le cas général, la bonne pratique consiste à décorer le wrapper avec `functools.wraps` pour conserver les métadonnées de la fonction d'origine (nom, docstring)
    * les décorateurs peuvent être paramétrés, on rajoute une fonction qui va générer le décorateur qui nous intéresse